In [1]:
# General imports
import numpy as np
import re
import sympy as sp
import json

# Pre-defined ansatz circuit and operator class for Hamiltonian
from qiskit.circuit.library import EfficientSU2, TwoLocal, RealAmplitudes
from qiskit.quantum_info import SparsePauliOp

# SciPy minimizer routine
from scipy.optimize import minimize, differential_evolution, basinhopping

# Plotting functions
import matplotlib.pyplot as plt

# runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_aer import AerSimulator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

aer_sim = AerSimulator()
pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=3)

In [2]:
def create_matrix(cut_off, type):
    # Initialize a zero matrix of the specified size
    matrix = np.zeros((cut_off, cut_off), dtype=np.complex128)
    
    # Fill the off-diagonal values with square roots of integers
    for i in range(cut_off):
        if i > 0:  # Fill left off-diagonal
            if type == 'q':
                matrix[i][i - 1] = (1/np.sqrt(2)) * np.sqrt(i)  # sqrt(i) for left off-diagonal
            else:
                matrix[i][i - 1] = (1j/np.sqrt(2)) * np.sqrt(i)

        if i < cut_off - 1:  # Fill right off-diagonal
            if type == 'q':
                matrix[i][i + 1] = (1/np.sqrt(2)) * np.sqrt(i + 1)  # sqrt(i + 1) for right off-diagonal
            else:
                matrix[i][i + 1] = (-1j/np.sqrt(2)) * np.sqrt(i + 1)

    return matrix

In [5]:
# Function to calculate the Hamiltonian
def calculate_Hamiltonian(cut_off):
    # Generate the position (q) and momentum (p) matrices
    q = create_matrix(cut_off, 'q')  # q matrix
    p = create_matrix(cut_off, 'p')  # p matrix
    
    #fermionic identity
    I_f = np.eye(2)

    # Calculate q^2 for potential terms
    q2 = np.dot(q, q)
    
    # Superpotential derivatives
    W_prime = q + q2 + np.eye(cut_off)  # W'(q) = q + q^2 + 1
    W_double_prime = np.eye(cut_off) + 2 * q  # W''(q) = 1 + 2q

    # Kinetic term: p^2
    p2 = np.dot(p, p)

    # Commutator term [b^†, b] = -Z
    Z = np.array([[1, 0], [0, -1]])  # Pauli Z matrix for fermion number
    commutator_term = np.kron(Z, W_double_prime)

    # Construct the block-diagonal kinetic term (bosonic and fermionic parts)
    # Bosonic part is the same for both, hence we use kron with the identity matrix
    kinetic_term = np.kron(I_f, p2)

    # Potential term (W' contribution)
    potential_term = np.kron(I_f, np.dot(W_prime, W_prime))

    # Construct the full Hamiltonian
    H_SQM = 0.5 * (kinetic_term + potential_term + commutator_term)

    return H_SQM

In [95]:
H = calculate_Hamiltonian(16)

In [96]:
sympy_matrix = sp.Matrix(H)
sp.pprint(sympy_matrix, use_unicode=True)

⎡      2.375        2.47487373415292  1.76776695296637  0.866025403784438  0.6 ↪
⎢                                                                              ↪
⎢2.47487373415292        5.875              5.0         4.28660704987056   1.7 ↪
⎢                                                                              ↪
⎢1.76776695296637         5.0              10.875       7.96084166404533   7.7 ↪
⎢                                                                              ↪
⎢0.866025403784438  4.28660704987056  7.96084166404533       17.375        11. ↪
⎢                                                                              ↪
⎢0.612372435695794  1.73205080756888  7.79422863405995  11.3137084989848       ↪
⎢                                                                              ↪
⎢        0          1.36930639376291  2.73861278752583  12.2983738762488   15. ↪
⎢                                                                              ↪
⎢        0                 0

In [97]:
hamiltonian = SparsePauliOp.from_operator(H)
ansatz = RealAmplitudes(num_qubits=hamiltonian.num_qubits, reps=1)
ansatz_isa = pm.run(ansatz)
hamiltonian_isa = hamiltonian.apply_layout(layout=ansatz_isa.layout)

In [98]:
hamiltonian

SparsePauliOp(['IIIII', 'IIIIX', 'IIIXI', 'IIIXX', 'IIIXZ', 'IIIYY', 'IIIZI', 'IIIZX', 'IIIZZ', 'IIXII', 'IIXIX', 'IIXIZ', 'IIXXI', 'IIXXX', 'IIXXZ', 'IIXYY', 'IIXZI', 'IIXZX', 'IIXZZ', 'IIYIY', 'IIYXY', 'IIYYI', 'IIYYX', 'IIYYZ', 'IIYZY', 'IIZII', 'IIZIX', 'IIZIZ', 'IIZXI', 'IIZXX', 'IIZXZ', 'IIZYY', 'IIZZI', 'IIZZX', 'IIZZZ', 'IXXII', 'IXXIX', 'IXXIZ', 'IXXXI', 'IXXXX', 'IXXXZ', 'IXXYY', 'IXXZI', 'IXXZX', 'IXXZZ', 'IXYIY', 'IXYXY', 'IXYYI', 'IXYYX', 'IXYYZ', 'IXYZY', 'IYXIY', 'IYXXY', 'IYXYI', 'IYXYX', 'IYXYZ', 'IYXZY', 'IYYII', 'IYYIX', 'IYYIZ', 'IYYXI', 'IYYXX', 'IYYXZ', 'IYYYY', 'IYYZI', 'IYYZX', 'IYYZZ', 'IZIII', 'IZIIX', 'IZIIZ', 'IZIXI', 'IZIXX', 'IZIXZ', 'IZIYY', 'IZIZI', 'IZIZX', 'IZIZZ', 'IZXII', 'IZXIX', 'IZXIZ', 'IZXXI', 'IZXXX', 'IZXXZ', 'IZXYY', 'IZXZI', 'IZXZX', 'IZXZZ', 'IZYIY', 'IZYXY', 'IZYYI', 'IZYYX', 'IZYYZ', 'IZYZY', 'IZZII', 'IZZIX', 'IZZIZ', 'IZZXI', 'IZZXX', 'IZZXZ', 'IZZYY', 'IZZZI', 'IZZZX', 'IZZZZ', 'ZIIII', 'ZIIIX', 'ZIIXX', 'ZIIYY', 'ZIIZX', 'ZIXXX', 'ZIX

In [99]:
cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (EstimatorV2): Estimator primitive instance
        cost_history_dict: Dictionary for storing intermediate results

    Returns:
        float: Energy estimate
    """
    pub = (ansatz, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    energy = result[0].data.evs[0]

    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = params
    cost_history_dict["cost_history"].append(energy)
    print(f"Iters. done: {cost_history_dict['iters']} [Current cost: {energy}]")

    return energy

In [100]:
num_params = ansatz.num_parameters
x0 = 0.025 * np.pi * np.random.random(num_params)

In [101]:
energies = []
x_values = []

for i in range(50):

    print("####################")
    print(f"Session loop {i}")

    with Session(backend=aer_sim) as session:
        estimator = Estimator(mode=session)
        estimator.options.default_shots = 10000
        #estimator.options.optimization_level = 1

        res = basinhopping(
            cost_func,
            x0,
            minimizer_kwargs={'method': "COBYLA",
                            "args": (ansatz_isa, hamiltonian_isa, estimator),
                            "options": {'maxiter':1000}},
                            #"tol": '0.00001'},
            niter = 3,
            T = 0.25
        )
        energies.append(res.fun)
        x_values.append(res.x)

####################
Session loop 0
Iters. done: 1 [Current cost: 1.944009156634375]
Iters. done: 2 [Current cost: 7.928894645830159]
Iters. done: 3 [Current cost: 15.201717796189373]
Iters. done: 4 [Current cost: 38.846974441812584]
Iters. done: 5 [Current cost: 23.13340891619205]
Iters. done: 6 [Current cost: 2.6027386040980005]
Iters. done: 7 [Current cost: 6.295916433059103]
Iters. done: 8 [Current cost: 7.240629360904279]
Iters. done: 9 [Current cost: 8.133407569372649]
Iters. done: 10 [Current cost: 20.349878109279658]
Iters. done: 11 [Current cost: 2.8587943621835943]
Iters. done: 12 [Current cost: 18.708308133683975]
Iters. done: 13 [Current cost: 7.585450705299406]
Iters. done: 14 [Current cost: 1.1657562907669956]
Iters. done: 15 [Current cost: 4.120154103577991]
Iters. done: 16 [Current cost: 5.620712620268165]
Iters. done: 17 [Current cost: 2.649691784013449]
Iters. done: 18 [Current cost: 4.988475253519638]
Iters. done: 19 [Current cost: 2.185282082174313]
Iters. done: 20 

In [102]:
energies

[2.3187345033511084,
 0.16067508467206604,
 2.3622324975922817,
 1.0423695622559224,
 1.2211909246214843,
 -0.09261545349198984,
 1.5830471716579892,
 3.413149671365558,
 0.9301625013039987,
 3.482535724046163,
 2.1634521083443228,
 2.468121630918058,
 1.8093086839761405,
 2.4938696712356365,
 2.685371123871479,
 2.3782628023355215,
 1.7946778969086041,
 1.3715548582556503,
 1.5589505116597666,
 2.9334933881591354,
 1.5238085683229612,
 3.7586150537720493,
 0.6888138748240612,
 2.3513936431601334,
 2.80188935848987,
 2.8352270246774083,
 1.9130522702001986,
 1.8443955212525016,
 2.6243238418967363,
 2.1774002458045754,
 0.17253703489386188,
 2.1126504516676428,
 2.1577656000771532,
 0.9594377329030103,
 2.597679163166549,
 1.3858719596322735,
 1.150003697704348,
 2.6204567125212805,
 2.0934696546894,
 0.02445538503581633,
 2.0711595275096006,
 1.5843025103713777,
 2.167572337928828,
 2.8030805109186874,
 1.349564274235753,
 2.423066926608854,
 2.032086472990391,
 2.8592392979323646,
 3

In [103]:
run = {
    'potential': 'DW',
    'cutoff': 16,
    'ansatz': 'RealAmplitudes',
    'num_sessions': 50,
    'backend': 'aer_simulator',
    'estimator_shots': 10000,
    'min_function': {'name': 'basinhopping',
                     'args': {'minimizer_kwargs': {'method': "COBYLA",
                                                    "args": ('ansatz_isa', 'hamiltonian_isa', 'estimator'),
                                                    "options": {'maxiter':1000}},   
                                'niter': 3,
                                'T': 0.25}
                    },
    'results': energies,
    'x_values': [x.tolist() for x in x_values]
}

In [104]:
path = r"C:\Users\Johnk\OneDrive\Desktop\PhD 2024\Quantum Computing Code\Quantum-Computing\Qiskit\SUSY VQE\Files\DW\DW_16.json"
# Save the variable to a JSON file
with open(path, 'w') as json_file:
    json.dump(run, json_file, indent=4)

In [105]:
n_values = [2, 4, 8, 16]
data_dict = {}

base_path = r"C:\Users\Johnk\OneDrive\Desktop\PhD 2024\Quantum Computing Code\Quantum-Computing\Qiskit\SUSY VQE\Files\DW\DW_{}.json"

for n in n_values:
    file_path = base_path.format(n)
    with open(file_path, 'r') as json_file:
        data_dict[f'c{n}'] = json.load(json_file)

In [106]:
for c in data_dict.keys():
    res = [x for x in data_dict[c]['results']]
    print(c)
    print("Min: ", min(res))
    print("Avg: ", np.mean(res))

c2
Min:  0.35723304703363135
Avg:  0.35747198729257645
c4
Min:  0.8466113822787331
Avg:  0.9510947408612955
c8
Min:  0.8260088587735414
Avg:  1.2667198787707088
c16
Min:  -0.09261545349198984
Avg:  1.9696552644683978


In [45]:
energies

[3.3952502800489377, 3.2945954147948715, 1.3243686262758112]

In [29]:
energies

[1.1648080973839756, 0.9636035894839348, 0.9100042458227151]

In [21]:
energies

[0.9068748173493216, 0.8579907239225715, 0.9248028773374947]

In [12]:
energies

[0.3579957887334283, 0.35731588974610595, 0.35723304703363135]